TASK 1

In [223]:
# Part 1
from Utils import timeout
import math
@timeout
def problem1_inversion(n_samples=1):
# Distribution from part 1
# write the code in this function to produce samples from the distribution␣
# ,→in the assignment
# Make sure you choose a good sampling distribution to avoid unnecessary␣
# ,→rejections
# Return a numpy array of length n_samples
    def function(x):
        if x <= 0:
            return 0 
        elif 0 < x < 1:
            return (math.exp(x**2)-1) / (math.exp(1)-1)
        else:
            return 1


# return xx

TASK 2

In [224]:
import numpy as np
import pandas as pd
file_path = "data/spam.csv"
data = pd.read_csv(file_path, encoding='latin1')
data


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [225]:
data = data[['v1', 'v2']]
data.columns = ['label', 'message']
data


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [226]:
# data['label'].map[{'ham':0, 'spam': 1}]
data['label']= data['label'].map({'ham':0, 'spam': 1})
data


C:\Users\saber\AppData\Local\Temp\ipykernel_19516\2297165419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label']= data['label'].map({'ham':0, 'spam': 1})


,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [227]:
problem3_X_HELPER = data['message'].apply(
    lambda x: [
        int('free' in x.lower()),
        int('prize' in x.lower()),
        int('win' in x.lower())
    ]
).to_list()

In [228]:
from sklearn.model_selection import train_test_split

# Part 1
problem2_X = np.array(problem3_X_HELPER)
problem2_Y = np.array(data['label'])

# First split (60% training, 40% test)
problem2_X_train, problem2_X_test, problem2_Y_train, problem2_Y_test = train_test_split(problem2_X, problem2_Y, test_size=0.4, random_state=42)

# Second split (33% of 60% for calibration, i.e., 20% of total data)
problem2_X_train, problem2_X_calib, problem2_Y_train, problem2_Y_calib = train_test_split(problem2_X_train, problem2_Y_train, test_size=0.33, random_state=42)


# problem2_X_train = XXX
# problem2_X_calib = XXX
# problem2_X_test = XXX
# problem2_Y_train = XXX
# problem2_Y_calib = XXX
# problem2_Y_test = XXX
print(problem2_X_train.shape,problem2_X_calib.shape,problem2_X_test.shape,problem2_Y_train.shape,problem2_Y_calib.shape,problem2_Y_test.shape)

(2239, 3) (1104, 3) (2229, 3) (2239,) (1104,) (2229,)


In [229]:
# Part 2

class ProportionalSpam(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
    
    # define the objective/cost/loss function we want to minimise
    def loss(self,X,Y,coeffs):
        beta_0 = coeffs[0]            # Intercept
        beta = coeffs[1:]             # Weights

        # Logistic function
        G = lambda z: 1 / (1 + np.exp(-z))

        # Compute probabilities
        p = G(np.dot(X, beta) + beta_0)

        # Binary cross-entropy loss
        return -np.mean(Y * np.log(p + 1e-9) + (1 - Y) * np.log(1 - p + 1e-9))
    
#trainning
    def fit(self,X,Y):
        import numpy as np
        from scipy import optimize

        #Use the f above together with an optimization method from scipy
        #to find the coefficients of the model
        opt_loss = lambda coeffs: self.loss(X,Y,coeffs)
        initial_arguments = np.zeros(shape=X.shape[1]+1)
        self.result = optimize.minimize(opt_loss, initial_arguments,method='cg')
        self.coeffs = self.result.x
#last step ,prediction  
    def predict(self,X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            G = lambda x: np.exp(x)/(1+np.exp(x))
            return np.round(10*G(np.dot(X,self.coeffs[1:])+self.coeffs[0]))/10 # This rounding is to help you with the calibration

In [230]:
# Part 3
from sklearn.tree import DecisionTreeRegressor
problem2_ps = ProportionalSpam()
problem2_ps.fit(problem2_X_train, problem2_Y_train)

problem2_X_pred = problem2_ps.predict(problem2_X_calib).reshape(-1, 1)


problem2_calibrator= DecisionTreeRegressor()
problem2_calibrator.fit(problem2_X_pred, problem2_Y_calib)

DecisionTreeRegressor()

In [231]:
# Part 4
# These are the predicted probabilities
final_test= problem2_ps.predict(problem2_X_test).reshape(-1,1)
 
problem2_final_predictions = problem2_calibrator.predict(final_test)
problem2_final_predictions
# In order to compute this loss we first need to convert the predicted␣
# ,→probabilities to a decision
# recall the Bayes classifier?
# problem2_01_loss = XXX
# Recall the interval is given as a tuple (a,b) or a list [a,b]
# problem2_interval = XXX

array([0.07917889, 0.07917889, 0.07917889, ..., 0.07917889, 0.07917889,
       0.07917889])

2023 problem 3

In [232]:
import numpy as np
import pandas as pd
file_path = " data/visits_clean.csv"
problem3_df = pd.read_csv("data/visits_clean.csv", sep = ' ')
problem3_df

,ofp,ofnp,opp,opnp,emr,hosp,exclhlth,poorhlth,numchron,adldiff,...,midwest,west,age,male,married,school,faminc,employed,privins,medicaid
0,5,0,0,0,0,1,0,0,2,0,...,0,0,6.9,1,1,6,2.881000,1,1,0
1,1,0,2,0,2,0,0,0,2,0,...,0,0,7.4,0,1,10,2.747800,0,1,0
2,13,0,0,0,3,3,0,1,4,1,...,0,0,6.6,0,0,10,0.653200,0,0,1
3,16,0,5,0,1,1,0,1,2,1,...,0,0,7.6,1,1,3,0.658800,0,1,0
4,3,0,0,0,0,0,0,0,2,1,...,0,0,7.9,0,1,6,0.658800,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,11,0,0,0,0,0,0,0,0,0,...,0,0,8.4,0,1,8,2.249700,0,1,0
4402,12,0,0,0,0,0,0,0,2,0,...,0,0,7.8,0,0,11,5.813200,0,1,0
4403,10,0,20,0,1,1,0,0,5,0,...,0,0,7.3,1,1,12,3.877916,0,1,0
4404,16,1,0,0,0,0,0,0,0,0,...,0,0,6.6,0,1,12,3.877916,0,1,0


In [233]:
# Step 2: Feature and target selection
features_to_exclude = ['ofp', 'ofnp', 'opp', 'opnp', 'emr']
X = problem3_df.drop(columns=features_to_exclude)
Y = problem3_df['ofp']

# Step 3: Split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Step 4: Preprocess data
def preprocess_data(X_train, X_test):
    X_train_encoded = pd.get_dummies(X_train)
    X_test_encoded = pd.get_dummies(X_test)
    X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
    return X_train_encoded, X_test_encoded

X_train, X_test = preprocess_data(X_train, X_test)

# Debugging output
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
# By using a structured approach like this, you can avoid many of the errors related to inconsistent data handling or unexpected edge cases. Let me know if you'd like further clarification!











X_train shape: (3524, 16)
Y_train shape: (3524,)


our y here is ofp 

NO categorical data 

In [236]:
# is identifying the categorical (non-numeric) columns in a DataFrame
s = (X_train.dtypes== 'object')
object_cols = list(s[s].index)
object_cols

[]

In [237]:
from scipy.special import factorial
from scipy import optimize

class PoissonRegression(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
    
    def fit(self,X,Y):
        import numpy as np
        from scipy import optimize
        #OBS LOSS FUNCTION IMPELEMENTAION 
        # define the objective/cost/loss function we want to minimize
        def loss(coeffs):
            # The parameter lambda for the given X and the proposed values 
            # of the coefficients, here coeff[:-1] represent alpha 
            # and coeff[-1] represent beta
            lam = np.exp(np.dot(X,coeffs[:-1])+coeffs[-1])

            # use the Y variable that is available here to definr 
            # this Y and for this parameter lam defined above
            loss_pr = - np.sum((np.subtract(np.multiply(Y, np.log(lam)), lam))) #+ np.log(factorial(y)) don't need the factorial
            
            return loss_pr
        
        #Use the loss above together with an optimization method from scipy
        #to find the coefficients of the model
        #this is prepared for you below

        initial_arguments = np.zeros(shape=X.shape[1]+1) # initial guess as 0
        self.result = optimize.minimize(loss, initial_arguments,method='cg')
        self.coeffs = self.result.x
    
    def predict(self,X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            return np.exp(np.dot(X,self.coeffs[:-1])+self.coeffs[-1])


In [238]:
print("X_train shape:", X_train.shape)  # Expected: (n_samples, n_features)
print("Y_train shape:", Y_train.shape)  # Expected: (n_samples,)


X_train shape: (3524, 16)
Y_train shape: (3524,)


In [239]:
# Initialize your PoissonRegression model
problem3_model = PoissonRegression()
problem3_model.fit(X_train,Y_train)

In [244]:
from sklearn.metrics import mean_squared_error

y_pred = problem3_model.predict(X_test)

print(problem3_model.result)
print(np.mean(y_pred))
print(mean_squared_error(y_pred, Y_test)) # <--- MSE

 message: Desired error not necessarily achieved due to precision loss.
 success: False
  status: 2
     fun: -16526.94251505277
       x: [ 1.590e-01 -3.762e-01 ...  2.654e-01  1.334e+00]
     nit: 686
     jac: [ 9.766e-04 -9.766e-04 ...  2.197e-03 -4.883e-04]
    nfev: 26777
    njev: 1487
5.746680496788654
43.8491425046477


1. irreduciable: all states are interconnected 
2. aperiodic : there are self loops 
3. periodic : at each state we have zero probability 
4. stationarey distribution: exists if the M is irreduciable and aperiodic 
5. reversibility: M must be irreduciable and aperiodic and the floe of probability between any two states must be symmetric 

In [240]:
# Function to compute the stationary distribution
def compute_stationary_distribution(matrix):
    # Solve πP = π subject to sum(π) = 1
    n = matrix.shape[0]
    A = np.transpose(matrix) - np.eye(n)
    A[-1, :] = 1  # Replace last row to enforce sum(π) = 1
    b = np.zeros(n)
    b[-1] = 1  # Corresponding value for the constraint
    stationary_distribution = np.linalg.solve(A, b)
    return stationary_distribution

def check_reversibility(matrix, stationary_distribution):
    """
    Check if the Markov chain satisfies the detailed balance condition.
    """
    n = matrix.shape[0]
    reversible = True
    for i in range(n):
        for j in range(n):
            if not np.isclose(stationary_distribution[i] * matrix[i, j],
                              stationary_distribution[j] * matrix[j, i]):
                reversible = False
                break
    return reversible

## The singular value decomposition (SVD) of the matrix 𝑀


In [ ]:
import numpy as np

# Define the matrix
M = np.array([[1, 1],
              [0, 3],
              [3, 0]])

# Compute the Singular Value Decomposition
U, singular_values, Vt = np.linalg.svd(M)

# Construct the diagonal matrix D with singular values
D = np.zeros_like(M, dtype=float)
np.fill_diagonal(D, singular_values)

# Print the results
rank = np.linalg.matrix_rank(M)
V = Vt.T  # Transpose of Vt gives V

print("Rank of the matrix M:", rank)
print("Matrix of singular vectors V:\n", V)
print("Diagonal matrix D:\n", D)
print("Matrix of right singular vectors U:\n", U)

# Verify the decomposition
M_reconstructed = U @ D @ V.T
print("Reconstructed M from U*D*V.T:\n", M_reconstructed)
print("Decomposition valid:", np.allclose(M, M_reconstructed))